In [ ]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

import re
import scipy
import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import display, HTML
from IPython.html.widgets import interact, interactive, fixed
from sqlalchemy import create_engine

%matplotlib inline

# Set up our MySQL connection with SQL Alchemy (this helps us to read directly into Pandas DataFrames

In [ ]:
engine = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'
                      .format('jovyan', 
                             getpass.getpass('Enter MySQL password for jovyan'),
                             'mysql',
                             '3306',
                             'mimic2'))

# Before we move ahead, we will do some counts of patients, admissions and notes to ensure connectivity and also get a sense of the dataset

In [ ]:
display(pd.read_sql_query('SELECT count(*) as PatientCount from d_patients', engine))
display(pd.read_sql_query('SELECT count(*) as AdmissionCount from admissions', engine))
display(pd.read_sql_query('SELECT count(*) as NoteCount from noteevents', engine))

# MIMIC-II (and MIMIC-III) has tables for Admissions, ICD-9 codes, notes and many other pieces of data

In [ ]:
display(pd.read_sql_query('SELECT * from admissions LIMIT 5', engine))

In [ ]:
display(pd.read_sql_query('SELECT * from icd9 LIMIT 5', engine))

In [ ]:
display(pd.read_sql_query('SELECT * from noteevents LIMIT 5', engine))

In [ ]:
# before pulling note text, let's get a distribution of how many RADIOLOGY reports
# typically exist per admission
visit_rad_report_count_query = """
SELECT
n.hadm_id
,count(*) as rad_note_count
FROM d_patients p
INNER JOIN noteevents n
    ON n.subject_id = p.subject_id
WHERE 
    Category = 'RADIOLOGY_REPORT' 
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND n.hadm_id IS NOT NULL
GROUP BY n.hadm_id
ORDER BY count(*) DESC
"""
visit_rad_report_count_df = pd.read_sql_query(visit_rad_report_count_query, engine)
display(visit_rad_report_count_df.head(10))

In [ ]:
rad_note_counts = visit_rad_report_count_df['rad_note_count'].values
scipy.stats.describe(rad_note_counts)

# Notes in MIMIC have a category (e.g. "RADIOLOGY_REPORT") and within the text there are often "sub categories" on the second line of the file.  Pulling the appropriate sub categories as a few "like" statements does the job, but it is worth looking at some of these on your own

In [ ]:
# before pulling note text, let's get a distribution of how many RADIOLOGY reports
# typically exist per admission
visit_rad_report_count_query = """
SELECT
n.hadm_id
,count(*) as rad_note_count
FROM d_patients p
INNER JOIN noteevents n
    ON n.subject_id = p.subject_id
WHERE 
    Category = 'RADIOLOGY_REPORT' 
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND n.hadm_id IS NOT NULL
GROUP BY n.hadm_id
ORDER BY count(*) DESC
"""
visit_rad_report_count_df = pd.read_sql_query(visit_rad_report_count_query, engine)
display(visit_rad_report_count_df)

# Some patients have only one radiology report but several have multiple.  This graph looks at that distribution

In [ ]:
rad_note_count_grouping = visit_rad_report_count_df.groupby('rad_note_count').size()
#display(rad_note_count_grouping)

note_count_bins = rad_note_count_grouping.index.values
#print(note_count_bins)
note_frequencies = rad_note_count_grouping.values
#print(note_frequencies)

fig = plt.figure(figsize=(16, 8))

plt.xlabel('Total Radiology Chest X-Ray Notes per visit')
plt.ylabel('Total Visits')

plt.bar(note_count_bins, note_frequencies)

# Finally we can pull these into a frame...

In [ ]:
# now let's pull a frame of all the FIRST (sorted by text which begins with date) CHEST X-RAY notes
chest_xray_note_query = """
SELECT
subject_id
,hadm_id
,LTRIM(RTRIM(text)) as text
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND subject_id is not NULL
    AND hadm_id is not NULL
"""
chest_xray_note_df = pd.read_sql_query(chest_xray_note_query, engine)
display(chest_xray_note_df)

# ... and then set up some widgets to be able to drag back and forth between the set to display them easily

In [ ]:
# This function let's us iterate through all documents and view the markup
def view_documents(reports):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(reports)-1))
    def _view_document(i):
        report_html = reports[i].replace('\n', '<br>')
        
        display(HTML(report_html))
        
chest_xray_list = list(chest_xray_note_df['text'].values)
view_documents(chest_xray_list)

# NOTE : It be useful to use these chest x-ray radiology reports to get an idea of some of the language in these reports
## For example, it could be useful to do a word count across this dataset
## It could also be interesting to do a count of most frequent n-grams from this set

# This may be worth coming back to later as we we our group projects